<a href="https://colab.research.google.com/github/IshanSFernando/Real-and-Synthetic-image-classification/blob/main/Resnet_150_Model_Evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import tqdm
import pickle
import os
from os import walk
import pandas as pd
from keras.models import load_model

In [3]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# *Raid Data Set Reading*

In [45]:
AUTOTUNE = tf.data.AUTOTUNE
def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, [224, 224])
def get_label(file_path):
  # Convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[index[0]] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

def process_path(file_path):
  label = [1,0]
  if get_label(file_path)==1:
    label = [0,1]
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label
def get_images(file_path):
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img

def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.batch(32)
  # ds = ds.shuffle(buffer_size=1000)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

def fileRead(dir_path,all):
  for (dir_path2, dir_names, file_names) in walk(dir_path):
    if len(file_names)>0:
      print(dir_path)
      for i in tqdm.tqdm(file_names,total=len(file_names)):
        all.append(dir_path+'/'+i)        
    if len(dir_names)>0:
      length = len(dir_names)
      for path in range(length-1,-1,-1):
        path_name = dir_names.pop()
        fileRead(dir_path+'/'+path_name,all)
def mainFileRead(dir_path):
  all=[]
  fileRead(dir_path,all)
  return all

In [46]:
#Make sure directory names are correct, otherwis results will be wrong
class_names = ['real', 'synthetic']

Directory_to_parent_folder = "/content/drive/MyDrive/VIP_CUP/general_data_set/test/v1"
#Directory_to_parent_folder = '/content/drive/MyDrive/VIP_CUP/Inpainting/Test/version_1'
imageList = mainFileRead(Directory_to_parent_folder)
# print(imageList[0])
parts = tf.strings.split(imageList[0], os.path.sep)
# print(parts)
print('-------------')
a = np.where(parts == class_names[0])
b = np.where(parts == class_names[1])
if len(a[0])>0:
  index = a[0] 
  print('a',a)
elif (len(b[0])>0):
  index = b[0]
  print('b',b)
else:
  print('Directory Error')
print('This nuber shuld be checked-',index[0])
print(parts[index[0]])

/content/drive/MyDrive/VIP_CUP/general_data_set/test/v1/synthetic/gan3


100%|██████████| 59/59 [00:00<00:00, 228709.74it/s]


/content/drive/MyDrive/VIP_CUP/general_data_set/test/v1/synthetic/glide


100%|██████████| 400/400 [00:00<00:00, 1007640.60it/s]


/content/drive/MyDrive/VIP_CUP/general_data_set/test/v1/synthetic/inpainting


...

/content/drive/MyDrive/VIP_CUP/general_data_set/test/v1/real/inpainting


100%|██████████| 1034/1034 [00:00<00:00, 751891.53it/s]


/content/drive/MyDrive/VIP_CUP/general_data_set/test/v1/real/gan2


100%|██████████| 102/102 [00:00<00:00, 412157.04it/s]

-------------
b (array([8]),)
This nuber shuld be checked- 8
tf.Tensor(b'synthetic', shape=(), dtype=string)


In [47]:
image_count = len(imageList)
list_ds = tf.data.Dataset.from_tensor_slices(imageList)
# val_size = int(image_count * 0.2)
# train_ds = list_ds.skip(val_size)
# val_ds = list_ds.take(val_size)

In [48]:
print('Number of Images',tf.data.experimental.cardinality(list_ds).numpy())
data_set = configure_for_performance(list_ds.map(process_path, num_parallel_calls=AUTOTUNE))
data_set

Number of Images 2143


<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.int32, name=None))>

#*Model Loading*

In [49]:
###### File name of the model #####
# model1= 'glide-resnet.pkl'
# dir = '/content/drive/MyDrive/VIP_CUP/Models/'
# dir = '/content/drive/MyDrive/VIP_CUP/Models/submission-v1.0/Models/'

# gan2 = load_model(dir+'gan2v2-resnet.h5')
# gan3 = load_model('/content/drive/MyDrive/VIP_CUP/Models/gan3v2-resnet.h5')
# glide = load_model(dir+'glide-resnet.h5')
# impainting = load_model(dir+'impainting-resnet.h5')
# transformer = load_model(dir+'Transform-resnet.h5')

general_model = load_model('/content/drive/MyDrive/VIP_CUP/Models/resnet50.h5')

#*Model Evaluating*

In [ ]:
# Do not run
# result = model.evaluate(data_set,batch_size=64)
# print('Loss =',result[0],'Accuracy =',result[1])

In [ ]:
def predict1(prediction1,prediction2,prediction3,prediction4,prediction5):
  if prediction1[0][0]<prediction1[0][1] or prediction2[0][0]<prediction2[0][1] or prediction3[0][0]<prediction3[0][1] or prediction4[0][0]<prediction4[0][1] or prediction5[0][0]<prediction5[0][1]:
    logit=1
  else:
    logit=-1
  return logit 
#gan2, gan3, glide, impainting, transformer
# Gan3 - 0.6165
# glide- 0.9312
# gan2 - 0.6201
# inpainting - 0.9295
# tranaform - 0.8411

def predict(prediction1,prediction2,prediction3,prediction4,prediction5):
  accuracies=[0.6201,0.6165,0.9312,0.9295,0.8411]
  accuracies=np.array(accuracies)
  accuracies=accuracies/accuracies.sum(axis=0,keepdims=1)
  real=prediction1[0][0]*accuracies[0]+prediction2[0][0]*accuracies[1]+prediction3[0][0]*accuracies[2]+prediction4[0][0]*accuracies[3]+prediction5[0][0]*accuracies[4]
  synthetic=prediction1[0][1]*accuracies[0]+prediction2[0][1]*accuracies[1]+prediction3[0][1]*accuracies[2]+prediction4[0][1]*accuracies[3]+prediction5[0][1]*accuracies[4]
  if real>synthetic:
    logit=-1
  else:
    logit=1  
  return logit  

In [50]:
def predict2(prediction1):
  real=prediction1[0][0]
  synthetic=prediction1[0][1]
  if real>synthetic:
    logit=-1
  else:
    logit=1  
  return logit  


In [51]:
singe_images = configure_for_performance(list_ds.map(get_images, num_parallel_calls=AUTOTUNE))
singe_images

<PrefetchDataset element_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None)>

In [52]:
imagescsv = np.array(imageList)
imagescsv = np.reshape(imagescsv, (-1, 1))
imagescsv = pd.DataFrame(imagescsv, columns=['filePath'])
imagescsv.to_csv('imagescsv.csv')
tab = pd.read_csv('imagescsv.csv')
labels = ['real', 'synthetic','Real', 'Synthetic']
predicton_list = []
for index, dat in tqdm.tqdm(tab.iterrows(), total=len(tab)):
  parts = tf.strings.split(dat['filePath'], os.path.sep)
  if (labels[0] in parts) or (labels[2] in parts):
    ans = -1
  elif (labels[1] in parts) or (labels[3] in parts):
    ans= +1
  else:
    print("Error")
    break
  predicton_list.append(ans)

100%|██████████| 2143/2143 [00:28<00:00, 75.24it/s]


In [53]:
tab['label']=predicton_list
print(tab)

      Unnamed: 0                                           filePath  label
0              0  /content/drive/MyDrive/VIP_CUP/general_data_se...      1
1              1  /content/drive/MyDrive/VIP_CUP/general_data_se...      1
2              2  /content/drive/MyDrive/VIP_CUP/general_data_se...      1
3              3  /content/drive/MyDrive/VIP_CUP/general_data_se...      1
4              4  /content/drive/MyDrive/VIP_CUP/general_data_se...      1
...          ...                                                ...    ...
2138        2138  /content/drive/MyDrive/VIP_CUP/general_data_se...     -1
2139        2139  /content/drive/MyDrive/VIP_CUP/general_data_se...     -1
2140        2140  /content/drive/MyDrive/VIP_CUP/general_data_se...     -1
2141        2141  /content/drive/MyDrive/VIP_CUP/general_data_se...     -1
2142        2142  /content/drive/MyDrive/VIP_CUP/general_data_se...     -1

[2143 rows x 3 columns]


In [56]:
for index, dat in tqdm.tqdm(tab.iterrows(), total=len(tab)): # for on images
    filename = dat['filePath']# filepath of an image
    test_image = tf.keras.utils.load_img(filename,target_size=(224,224))
    test_image = tf.keras.utils.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    #logit = predict(gan2.predict(test_image),gan3.predict(test_image),glide.predict(test_image),impainting.predict(test_image),transformer.predict(test_image))# TODO compute the logit for the image
    logit = predict2(general_model.predict(test_image))
    tab.loc[index,'logit'] = logit  # insert the logit in table
tab.to_csv('output_csv', index=False) 

100%|██████████| 2143/2143 [18:30<00:00,  1.93it/s]


In [57]:
out = pd.read_csv('output_csv')
print(out)

      Unnamed: 0                                           filePath  label  \
0              0  /content/drive/MyDrive/VIP_CUP/general_data_se...      1   
1              1  /content/drive/MyDrive/VIP_CUP/general_data_se...      1   
2              2  /content/drive/MyDrive/VIP_CUP/general_data_se...      1   
3              3  /content/drive/MyDrive/VIP_CUP/general_data_se...      1   
4              4  /content/drive/MyDrive/VIP_CUP/general_data_se...      1   
...          ...                                                ...    ...   
2138        2138  /content/drive/MyDrive/VIP_CUP/general_data_se...     -1   
2139        2139  /content/drive/MyDrive/VIP_CUP/general_data_se...     -1   
2140        2140  /content/drive/MyDrive/VIP_CUP/general_data_se...     -1   
2141        2141  /content/drive/MyDrive/VIP_CUP/general_data_se...     -1   
2142        2142  /content/drive/MyDrive/VIP_CUP/general_data_se...     -1   

      logit  
0      -1.0  
1      -1.0  
2       1.0  
3      

In [61]:
label = np.array(out['label'])
#label = label.astype(np.int32)
logit = np.array(out['logit'])
#logit = label.astype(np.int32)
eval = np.add(label,logit)

In [62]:
zero_count = np.count_nonzero(eval==0)
accu = (len(eval)-zero_count)/len(eval)
print('Accuracy;-',accu)

Accuracy;- 0.5730284647690154


In [63]:
true_postive = np.count_nonzero(eval==2)
true_negative= np.count_nonzero(eval==-2)
false_postive = np.count_nonzero(logit ==1)-np.count_nonzero(eval == 2)
false_negative = np.count_nonzero(logit ==-1)-np.count_nonzero(eval == -2)
length = len(eval)
print('true_postive-',true_postive/length,'true_negative-',true_negative/length,'false_negative-',false_negative/length,'false_postive-',false_postive/length)


true_postive- 0.06952869808679421 true_negative- 0.5034997666822212 false_negative- 0.377974801679888 false_postive- 0.04899673355109659
